In [1]:
from IPython.display import clear_output

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

import pandas as pd
import numpy as np

import json
import time
import mouse
import os
from os.path import exists


In [7]:
df_not_full = pd.read_excel('YM_not_enough.xlsx')
df_not_full

,lvl2_href,item_name,value
0,https://market.yandex.ru/catalog--smartfony/61808,"Телефон BQ 1411 Nano, 2 SIM, красный",843
1,https://market.yandex.ru/catalog--smartfony/61808,"Телефон Nokia 2700 Classic, 1 SIM, серый",700
2,https://market.yandex.ru/catalog--smartfony/61808,"Смартфон BQ 4030G Nice Mini 1/16 ГБ, micro SIM...",1 846
3,https://market.yandex.ru/catalog--smartfony/61808,"Телефон BQ 3586 Tank Max, 2 SIM, зеленый",3 190
4,https://market.yandex.ru/catalog--smartfony/61808,"Телефон MAXVI T100, 2 SIM, черный/красный",1 267
...,...,...,...
384270,https://market.yandex.ru//catalog--vitamin-d/5...,"Skin Hair & Nails Advanced Beauty капс., 120 м...",1 649
384271,https://market.yandex.ru//catalog--vitamin-d/5...,Animal Parade Gold Multi Assorted Flavors таб....,3 575
384272,https://market.yandex.ru//catalog--vitamin-d/5...,"АВС плюс гринс таб., 60 шт.",2 168
384273,https://market.yandex.ru//catalog--vitamin-d/5...,"NOW Eve Superior Women's Multi Vitamin, 180 ге...",4 100


In [6]:
df = pd.read_csv('YM_item_sample.csv', sep='\t', error_bad_lines=False)
df

C:\Users\lokri\AppData\Local\Temp\ipykernel_26988\690397680.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('YM_item_sample.csv', sep='\t', error_bad_lines=False)


,lvl2_href,item_name,value
0,https://market.yandex.ru/catalog--chekhly-dlia...,"Чехол для Samsung A52 / Чехол на Самсунг А52, ...",95
1,https://market.yandex.ru/catalog--chekhly-dlia...,Чехол накладка G-Case Carbon для Xiaomi Redmi ...,117
2,https://market.yandex.ru/catalog--chekhly-dlia...,Чехол накладка G-Case Slim Premium для Samsung...,132
3,https://market.yandex.ru/catalog--chekhly-dlia...,"Чехол Gel Plus Case для Apple iPhone 5/5S/SE, ...",99
4,https://market.yandex.ru/catalog--chekhly-dlia...,Чехол G-Case Carbon для Samsung Galaxy A32 4(G...,136
...,...,...,...
1855,https://market.yandex.ru/catalog--aksessuary-d...,нет данных,нет данных
1856,https://market.yandex.ru/catalog--aksessuary-d...,нет данных,нет данных
1857,https://market.yandex.ru/catalog--lezhaki-i-sh...,нет данных,нет данных
1858,https://market.yandex.ru/catalog--ruchnye-sado...,нет данных,нет данных


In [21]:
df_full = df_not_full.dropna().append(df).reset_index()
df_full

C:\Users\lokri\AppData\Local\Temp\ipykernel_26988\2531999077.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full = df_not_full.dropna().append(df).reset_index()


,index,lvl2_href,item_name,value
0,0,https://market.yandex.ru/catalog--smartfony/61808,"Телефон BQ 1411 Nano, 2 SIM, красный",843
1,1,https://market.yandex.ru/catalog--smartfony/61808,"Телефон Nokia 2700 Classic, 1 SIM, серый",700
2,2,https://market.yandex.ru/catalog--smartfony/61808,"Смартфон BQ 4030G Nice Mini 1/16 ГБ, micro SIM...",1 846
3,3,https://market.yandex.ru/catalog--smartfony/61808,"Телефон BQ 3586 Tank Max, 2 SIM, зеленый",3 190
4,4,https://market.yandex.ru/catalog--smartfony/61808,"Телефон MAXVI T100, 2 SIM, черный/красный",1 267
...,...,...,...,...
386129,1855,https://market.yandex.ru/catalog--aksessuary-d...,нет данных,нет данных
386130,1856,https://market.yandex.ru/catalog--aksessuary-d...,нет данных,нет данных
386131,1857,https://market.yandex.ru/catalog--lezhaki-i-sh...,нет данных,нет данных
386132,1858,https://market.yandex.ru/catalog--ruchnye-sado...,нет данных,нет данных


In [52]:
mask = df_full['item_name'].str.contains('чехол для тел', case=False, regex=True)
df_full[mask]['lvl2_href'].unique()

sorted(df_full['lvl2_href'].unique().tolist())

['https://market.yandex.ru//catalog--aksessuary-dlia-optiki/57658',
 'https://market.yandex.ru//catalog--aminokisloty/50494300',
 'https://market.yandex.ru//catalog--analnye-stimuliatory/18019244',
 'https://market.yandex.ru//catalog--audioknigi/55231',
 'https://market.yandex.ru//catalog--avtorskie-bukety/30584411',
 'https://market.yandex.ru//catalog--bdsm-atributika/18019245',
 'https://market.yandex.ru//catalog--biser-i-biseropletenie/18044618',
 'https://market.yandex.ru//catalog--chertezhnye-instrumenty/17984865',
 'https://market.yandex.ru//catalog--dezinfitsiruiushchie-sredstva/56057',
 'https://market.yandex.ru//catalog--doski-i-molberty-dlia-risovaniia/17985568',
 'https://market.yandex.ru//catalog--eroticheskie-stimuliatory/18019223',
 'https://market.yandex.ru//catalog--falloimitatory/18019221',
 'https://market.yandex.ru//catalog--futliary-dlia-ochkov/69061',
 'https://market.yandex.ru//catalog--igrovaia-valiuta/18059656',
 'https://market.yandex.ru//catalog--igry-dlia-pri

In [22]:
df_full.loc[3, 'value']

'3\u2009190'

In [24]:
import re

In [ ]:
re.

In [48]:
a = re.search(r'\d+(?=₽)', 'Цена с картой Яндекс Пэй:\n3799₽')
a.group(0)

'3799'

False

In [39]:
if df_full.loc[0,'value'].isdigit()

True

In [51]:
df_full.to_excel('YM_item_names_full.xlsx', index=False)

In [50]:
df_full['new_val'] = df_full['value'].str.replace('\u2009', '')
df_full.apply(lambda x: if )

,index,lvl2_href,item_name,value,new_val
0,0,https://market.yandex.ru/catalog--smartfony/61808,"Телефон BQ 1411 Nano, 2 SIM, красный",843,843
1,1,https://market.yandex.ru/catalog--smartfony/61808,"Телефон Nokia 2700 Classic, 1 SIM, серый",700,700
2,2,https://market.yandex.ru/catalog--smartfony/61808,"Смартфон BQ 4030G Nice Mini 1/16 ГБ, micro SIM...",1 846,1846
3,3,https://market.yandex.ru/catalog--smartfony/61808,"Телефон BQ 3586 Tank Max, 2 SIM, зеленый",3 190,3190
4,4,https://market.yandex.ru/catalog--smartfony/61808,"Телефон MAXVI T100, 2 SIM, черный/красный",1 267,1267
...,...,...,...,...,...
386129,1855,https://market.yandex.ru/catalog--aksessuary-d...,нет данных,нет данных,нет данных
386130,1856,https://market.yandex.ru/catalog--aksessuary-d...,нет данных,нет данных,нет данных
386131,1857,https://market.yandex.ru/catalog--lezhaki-i-sh...,нет данных,нет данных,нет данных
386132,1858,https://market.yandex.ru/catalog--ruchnye-sado...,нет данных,нет данных,нет данных


In [42]:
df_full[~df_full['value'].str.replace('\u2009', '').str.isdigit()]['value'].unique()

['Цена с картой Яндекс Пэй:\n3\u2009799\u2009₽',
 'Цена с картой Яндекс Пэй:\n4\u2009749\u2009₽',
 'Цена с картой Яндекс Пэй:\n2\u2009849\u2009₽',
 'Цена с картой Яндекс Пэй:\n2\u2009089\u2009₽',
 'Цена с картой Яндекс Пэй:\n3\u2009989\u2009₽',
 'Цена с картой Яндекс Пэй:\n1\u2009131\u2009₽',
 'Цена с картой Яндекс Пэй:\n1\u2009321\u2009₽',
 'Цена с картой Яндекс Пэй:\n751\u2009₽',
 'Цена с картой Яндекс Пэй:\n7\u2009591\u2009₽',
 'Цена с картой Яндекс Пэй:\n10\u2009441\u2009₽',
 'Цена с картой Яндекс Пэй:\n8\u2009541\u2009₽',
 'Цена с картой Яндекс Пэй:\n12\u2009816\u2009₽',
 'Цена с картой Яндекс Пэй:\n19\u2009941\u2009₽',
 'Цена с картой Яндекс Пэй:\n941\u2009₽',
 'Цена с картой Яндекс Пэй:\n1\u2009511\u2009₽',
 'Цена с картой Яндекс Пэй:\n846\u2009₽',
 'Цена с картой Яндекс Пэй:\n1\u2009416\u2009₽',
 'Цена с картой Яндекс Пэй:\n656\u2009₽',
 'Цена с картой Яндекс Пэй:\n17\u2009091\u2009₽',
 'Цена с картой Яндекс Пэй:\n16\u2009141\u2009₽',
 'Цена с картой Яндекс Пэй:\n24\u2009691\u2

In [43]:
df_full[df_full['value'] == '5\u2009630\u2009₽']

,index,lvl2_href,item_name,value
302048,302049,https://market.yandex.ru/catalog--tufli-i-lofe...,"Лоферы Respect, демисезонные, натуральная кожа...",5 630 ₽
